# Day 30: Human-in-the-Loop Workflow

In this lab, we will simulate a **Content Moderation** pipeline.
If the Model's confidence is low, the request will be escalated to a queue for human review (`HITLManager`)

In [ ]:
import sys
import os
import time

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.governance.hitl.py import HITLManager # Typo correction: logic should be from src.governance.hitl
from src.governance.hitl import HITLManager

## 1. Simulate Content and Confidence

We pretend we have processed 5 posts. Some are clearly safe/unsafe (high confidence), others are ambiguous (low confidence).

In [ ]:
# (Post Content, Model Prediction, Confidence Score)
pipeline_outputs = [
    ("This is a nice kitty.", "SafePrediction", 0.99),
    ("I hate everyone.", "UnsafePrediction", 0.95),
    ("I don't like Mondays.", "UnsafePrediction", 0.55), # Low confidence, sounds negative but maybe safe?
    ("He is annoying.", "UnsafePrediction", 0.60),
    ("Hello world.", "SafePrediction", 0.98)
]

# Manager configured to escalate if confidence < 0.8
manager = HITLManager(escalation_threshold=0.8)

## 2. Process Items

Route them through the manager.

In [ ]:
final_decisions = []

for content, pred, score in pipeline_outputs:
    review_id = manager.analyze_and_route(content, score)
    
    if review_id:
        print(f"[ESCALATED] '{content}' (Score: {score}) -> Review ID: {review_id}")
    else:
        print(f"[AUTO-APPROVED] '{content}' (Score: {score}) -> prediction accepted.")
        final_decisions.append((content, pred, "Auto"))

## 3. Human Review

The manager now has pending requests. In a real app, these would appear on a dashboard.

In [ ]:
pending = manager.get_pending_requests()
print(f"\nPending Reviews: {len(pending)}")

for req in pending:
    print(f"Reviewing: {req.content} (Reason: {req.reason})")
    # Simulate human feedback
    # Human decides: "I don't like Mondays" is Safe, "He is annoying" is Unsafe
    if "Mondays" in req.content:
        decision = "Approved (Overridden to Safe)"
        manager.submit_feedback(req.id, "Modify", "Marked as Safe")
    else:
        decision = "Confirmed Unsafe"
        manager.submit_feedback(req.id, "Approve", "Confirmed model prediction")
    
    final_decisions.append((req.content, decision, "Human"))

## 4. Final Logs

Show the final audit trail.

In [ ]:
import pandas as pd
df = pd.DataFrame(final_decisions, columns=["Content", "Outcome", "Actor"])
print(df)